In [ ]:
#export PATH=/data/Projects/phenomata/99.Tools/anaconda3/bin:$PATH
#source activate scanpy_1.9.1
#ipython --profile=vaging (in cm03)

from anndata import AnnData
import anndata
from scipy import sparse, io
import scipy
import pandas as pd
import scipy.io
import os
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors
matplotlib.use('TkAgg')
import numpy as np
import seaborn as sns
import math
import scanpy.external as sce
import scrublet as scr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
sns.set_theme(font="Arial", font_scale=1, style='ticks')
sc.settings.verbosity = 3
plt.rcParams['figure.figsize'] = (6,6)
plt.rc("axes.spines", top=False, right=False)
#plt.rcParams['font.family'] = 'sans-serif'
#plt.rcParams['font.sans-serif'] = 'Arial'
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#104e8b", "#ffdab9", "#8b0a50"])
batch_palette=['#689aff', '#fdbf6f', '#b15928']
%matplotlib
%autoindent

m01 = sc.read_10x_h5("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/01.Cell-Ranger/01month_filtered_feature_bc_matrix.h5")
m01.var_names_make_unique()

m10 = sc.read_10x_h5("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/01.Cell-Ranger/10months_filtered_feature_bc_matrix.h5")
m10.var_names_make_unique()

m20 = sc.read_10x_h5("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/01.Cell-Ranger/20months_filtered_feature_bc_matrix.h5")
m20.var_names_make_unique()

mito_genes = m01.var_names.str.startswith('mt-')
m01.obs['percent_mito'] = np.ravel(np.sum(m01[:, mito_genes].X, axis=1)) / np.ravel(np.sum(m01.X, axis=1))
m10.obs['percent_mito'] = np.ravel(np.sum(m10[:, mito_genes].X, axis=1)) / np.ravel(np.sum(m10.X, axis=1))
m20.obs['percent_mito'] = np.ravel(np.sum(m20[:, mito_genes].X, axis=1)) / np.ravel(np.sum(m20.X, axis=1))

for sample in [m01, m10, m20]:
    sce.pp.scrublet(sample, adata_sim=None, expected_doublet_rate=0.05, sim_doublet_ratio=2.0, stdev_doublet_rate=0.02, synthetic_doublet_umi_subsampling=1.0, knn_dist_metric='euclidean', n_prin_comps=30, verbose=True)
    
sc.pp.filter_cells(m01, min_counts=2000)
sc.pp.filter_cells(m01, min_genes=1500)

sc.pp.filter_cells(m10, min_counts=3000)
sc.pp.filter_cells(m10, min_genes=1500)

sc.pp.filter_cells(m20, min_counts=3000)
sc.pp.filter_cells(m20, min_genes=1500)

m01 = m01[m01.obs['percent_mito'] < 0.2]
m10 = m10[m10.obs['percent_mito'] < 0.2]
m20 = m20[m20.obs['percent_mito'] < 0.2]

integrated = AnnData.concatenate(m01, m10, m20, join='outer', batch_categories = ['m01', 'm10', 'm20'], index_unique = '-')
integrated.obs['Doublet'] = integrated.obs['predicted_doublet'].astype(str).astype('category')
integrated.obs[['Doublet', 'batch']].value_counts()
del integrated.obs['predicted_doublet']

sc.pp.filter_genes(integrated, min_cells=5) # 'n_cells' added in integrated.var 
integrated.layers["counts"] = integrated.X.copy()
integrated.raw = integrated

import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import anndata2ri
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython
%%R
library(scran)
library(dplyr)



%config InlineBackend.figure_format = 'retina'

adata_pp = integrated.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp) # works on anndata.X
sc.tl.pca(adata_pp, n_comps=15) ## 여기서 이 n_component의 숫자를 늘리면 size_factors를 estimation하는 데 도움이 될까?
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.5)
input_groups = adata_pp.obs['groups']
data_mat = integrated.X.T
%%R -i data_mat -i input_groups -o size_factors
size_factors = BiocGenerics::sizeFactors(computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups, min.mean=0.1))




del adata_pp
del data_mat

integrated.obs['size_factors'] = size_factors

integrated.X /= integrated.obs['size_factors'].values[:, None]
integrated.layers['scran'] = integrated.X # For cellphoneDB or CelChat maybe?
sc.pp.log1p(integrated) # works on anndata.X
integrated.X = scipy.sparse.csr_matrix(integrated.X)
integrated.raw = integrated ## ==> log transforamtion 된 것이 raw로 들어가게 됨.

test3 = integrated.copy()
test3.raw = test3
test3.layers['scran_log1p'] = test3.X

#del integrated

sc.pp.highly_variable_genes(test3)
test3.var['highly_variable'].value_counts() # 2,410 ==> 2021-08-10 # 2,513 ==> 2022-09-26

sc.pp.scale(test3, max_value=10) # tabula muris senis default (2021-08-10) # mean and std on adata.var
#sc.pp.scale(test3, zero_center=True, max_value=10, copy=False, layer=None, obsm=None)

cell_cycle_genes=[x.strip()[0] + x.strip()[1:].lower() for x in open("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/Database/regev_lab_cell_cycle_genes.txt")]
s_genes= cell_cycle_genes[:43]
g2m_genes= cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in test3.var_names]
sc.tl.score_genes_cell_cycle(test3, s_genes=s_genes, g2m_genes=g2m_genes)
"""
Used 'raw' attribute of adata (use_raw = True if .raw is present)
So, log-tranformed scran-normalized counts are put into score_genes_cell_cycle function
"""
sc.tl.pca(test3, n_comps=100, use_highly_variable=True, svd_solver='arpack')

sc.pl.pca_variance_ratio(test3, n_pcs=100, log=False)
sc.pl.pca(test3, legend_loc='right margin', size=8, add_outline=False, color_map='CMRmap', components=['1,2'], color=['batch'])

#sce.pp.bbknn default ==> n_pcs=50, neighbors_within_batch=3, trim=None, annoy_n_trees=10,
sce.pp.bbknn(test3, batch_key='batch', n_pcs=20, neighbors_within_batch=5, trim=None)
sc.tl.umap(test3, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
test3.uns['batch_colors'] = ['#689aff', '#fdbf6f', '#b15928']
sc.pl.umap(test3, add_outline=False, legend_loc='right margin', size=20, color=['batch'])

sc.tl.leiden(test3, resolution=0.5, key_added='leiden_r05') #### 0 ~ 13 ==> 2021-09-28
sc.tl.leiden(test3, resolution=1.0, key_added='leiden_r10')
sc.pl.umap(test3, add_outline=False, legend_loc='right margin', size=20, color=['batch', 'leiden_r05', 'leiden_r10'])

fig, axes = plt.subplots(1,3)
sc.pl.umap(test3, add_outline=False, legend_loc='right margin', size=20, groups=['m01'], title='1 month', ax=axes[0], color=['batch'])
sc.pl.umap(test3, add_outline=False, legend_loc='right margin', size=20, groups=['m10'], title='10 months', ax=axes[1], color=['batch'])
sc.pl.umap(test3, add_outline=False, legend_loc='right margin', size=20, groups=['m20'], title='20 months', ax=axes[2], color=['batch'])


### Cell-type annotation

In [ ]:
test3 = sc.read_h5ad("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3.h5ad")

sc.tl.rank_genes_groups(test3, 'leiden_r05', method='wilcoxon', corr_method='benjamini-hochberg', use_raw=True, pts=True) # key_added=''
sc.pl.rank_genes_groups(test3, n_genes=5, sharey=False)

leiden_to_consensus_celltype_dict = {'0': 'vSMC',
'1': 'vSMC',
'2': 'vSMC',
'3': 'FB',
'4': 'vSMC',
'5': 'EC',
'6': 'FB',
'7': 'EC',
'8': 'vSMC',
'9': 'FB',
'10': 'Bc',
'11': 'M\u03A6',
'12': 'Tc',
'13': 'Neuronal'}
test3.obs['Consensus_cell_type'] = test3.obs['leiden_r05'].map(lambda x: leiden_to_consensus_celltype_dict[x]).astype('category')
consensus_celltype_order = ('EC', 'vSMC', 'FB', 'Bc', 'M\u03A6', 'Tc', 'Neuronal')
test3.obs['Consensus_cell_type'] = test3.obs['Consensus_cell_type'].cat.reorder_categories(list(consensus_celltype_order), ordered=True)

leiden_to_detailed_celltype_dict = {'0': 'vSMC1',
'1': 'vSMC2',
'2': 'vSMC3',
'3': 'FB1',
'4': 'vSMC4',
'5': 'EC1',
'6': 'FB2',
'7': 'EC2',
'8': 'vSMC5',
'9': 'FB3',
'10': 'Bc',
'11': 'Mphage',
'12': 'Tc',
'13': 'Neuronal'}
test3.obs['detailed_celltype'] = test3.obs['leiden_r05'].map(lambda x: leiden_to_detailed_celltype_dict[x]).astype('category')
detailed_celltype_order = ('EC1', 'EC2', 'vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3', 'Bc', 'Mphage', 'Tc', 'Neuronal')
test3.obs['detailed_celltype'] = test3.obs['detailed_celltype'].cat.reorder_categories(list(detailed_celltype_order), ordered=True)

sc.pl.umap(test3, add_outline=False, legend_loc='right margin', color_map='viridis', color=['Klf4', 'batch', 'celltype'])

# There are numerous controversy that non-linear projection (e.g. UMAP/t-SNE) fails to represent true (in a sense) topological nature of the high-dimensional data.
# So it's best to first detect clusters (based on leiden clustering for example) and then look into those cluster using conventional similiarity measure such as hierarchical clustering 
ax = sc.pl.correlation_matrix(test3, groupby='detailed_celltype', show_correlation_numbers=True, dendrogram=True, ax=None, vmin=-1, vmax=1)

celltype_marker = {'EC': ['Pecam1', 'Cdh5', 'Vwf', 'Nos3'],
'vSMC': ['Acta2', 'Tagln', 'Cnn1', 'Cnn2'],
'FB': ['Dpt', 'Col1a1', 'Col5a1', 'Pdgfra'],
'Bc': ['Ighm', 'Cd19'],
'M\u03A6':['Cd14', 'Cd68'],
'Tc':['Cd3d', 'Cd3g'],
'Neuronal':['Mbp', 'Cnp']}

celltype_marker2 = {'EC': ['Pecam1', 'Cdh5', 'Vwf', 'Nos3'],
'vSMC': ['Acta2', 'Tagln', 'Cnn1', 'Cnn2'],
'FB': ['Dpt', 'Lum', 'Col1a1', 'Pdgfra'],
'Bc': ['Ighm', 'Cd19'],
'M\u03A6':['Cd14', 'Cd68'],
'Tc':['Cd3d', 'Cd3g'],
'Neuronal':['Mbp', 'Cnp']}
#sc.pl.matrixplot(test3, celltype_marker2, layer='magic', groupby='Consensus_cell_type', dendrogram=False, cmap=cmap, standard_scale='var', colorbar_title='Scaled\nexpression', swap_axes=True)

celltype_marker2_genes = list()
for key, value in celltype_marker2.items():
    celltype_marker2_genes += value


#sc.pl.matrixplot(test3, celltype_marker2, use_raw=True, groupby='Consensus_cell_type', dendrogram=False, cmap=cmap, standard_scale='var', colorbar_title='Scaled\nexpression', swap_axes=True)

ax_dict = sc.pl.matrixplot(test3, celltype_marker2, use_raw=True, groupby='Consensus_cell_type', dendrogram=False, cmap=cmap, standard_scale='var', colorbar_title='Scaled\nexpression', var_group_rotation=15, show=False)
ax_dict['mainplot_ax'].set_xticklabels(labels=celltype_marker2_genes, fontstyle='italic', rotation=30)
plt.xlabel("")
plt.tight_layout()

ax_dict = sc.pl.matrixplot(test3, celltype_marker2, use_raw=True, groupby='detailed_celltype', dendrogram=False, cmap=cmap, standard_scale='var', colorbar_title='Scaled\nexpression', var_group_rotation=15, show=False)
ax_dict['mainplot_ax'].set_xticklabels(labels=celltype_marker2_genes, fontstyle='italic', rotation=30)
plt.xlabel("")
plt.tight_layout()


#dp = sc.pl.dotplot(test3, celltype_marker, layer='magic', groupby='Annotated Cell Types', return_fig=True)
#dp.add_totals(size=1.5, color=['#393b79', '#9c9ede', '#b5cf6b', '#e7ba52', '#ad494a', '#7b4173', '#de9ed6']).legend(colorbar_title='log(SizeFactorNormlized+1)', width=1.5, show_size_legend=False, show_colorbar=False).style(cmap='Reds', dot_edge_color='black', dot_edge_lw=1, size_exponent=1.5, grid=True, x_padding=0.4, y_padding=0.6).swap_axes().show()


#### Fold enrichment (Odds ratio) of fine-grained subpopulation of all cells for each age group

In [ ]:
df = test3.obs[['batch', 'detailed_celltype']]
df_pivot = pd.crosstab(df['batch'], df['detailed_celltype'], normalize=False, margins=True)

# Percentage (intracluster normalization) heatmap 
detailed_celltype_order = ('EC1', 'EC2', 'vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3', 'Bc', 'Mphage', 'Tc', 'Neuronal')
df_pivot_percentage = pd.crosstab(df['batch'], df['detailed_celltype'], margins=False, normalize='columns')*100
df_pivot_percentage = df_pivot_percentage.reindex(columns=detailed_celltype_order)
sns.heatmap(df_pivot_percentage, vmin=0, vmax=100, cmap='viridis', annot=True, fmt='.2f')
plt.tight_layout()

# Odds ratio calculation
oddsratio_df, pvalue_df = list(), list()
for month in df_pivot.index[:-1]:
    oddsratio, pvalues = list(), list()
    for celltype in df_pivot.columns[:-1]:
#        table = np.array([ [df_pivot[celltype][month], df_pivot[celltype]['All'] - df_pivot[celltype][month] ], [df_pivot['All'][month], df_pivot['All']['All'] - df_pivot['All'][month]] ])
        table = np.array([ [df_pivot[celltype][month], df_pivot['All'][month] - df_pivot[celltype][month]], [df_pivot[celltype]['All'] - df_pivot[celltype][month], (df_pivot['All']['All'] - df_pivot['All'][month]) - (df_pivot[celltype]['All'] - df_pivot[celltype][month])] ])
        oddsr, p = fisher_exact(table, alternative='two-sided')
        oddsratio.append(oddsr)
        pvalues.append(p)
    oddsratio_df.append(oddsratio)
    pvalues = multipletests(pvals=pvalues, alpha=0.01, method='fdr_bh')[1]
    pvalue_df.append(pvalues)

Odds = pd.DataFrame(oddsratio_df, index=df_pivot.index[:-1], columns=df_pivot.columns[:-1])
Pvalues = pd.DataFrame(pvalue_df, index=df_pivot.index[:-1], columns=df_pivot.columns[:-1])

df_final = pd.concat([Odds, Pvalues], axis=0)
df_final

batch_palette= {'m01': '#689aff',
                'm10': '#fdbf6f',
                'm20': '#b15928'}
plt.rcParams['figure.figsize'] = (10,6)
ax = df_final.iloc[:3].T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(0.85, 0.95), frameon=False, prop={'size':15})
ax.set_ylabel('Odds Ratio')
ax.axhline(y=1.0, color="red", linewidth=1.0, linestyle='--')
plt.tight_layout()
sns.despine(ax=ax)
plt.rcParams['figure.figsize'] = (6,6) # back to the original parameter
#plt.savefig("FigureD.pdf")

# only for FB2 cluster
ax = df_final.iloc[:3, 8:9].T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(0.85, 0.95), frameon=False, prop={'size':15})
ax.set_ylabel('Odds Ratio')
ax.axhline(y=1.0, color="red", linewidth=1.0, linestyle='--')
plt.tight_layout()
sns.despine(ax=ax)

# Scale [0,1] using min-max normalization for each vSMC subcluster
df_odds_scale = (df_final.iloc[:3] - df_final.iloc[:3].min()) / ( df_final.iloc[:3].max() - df_final.iloc[:3].min() )
ax = df_odds_scale.sort_values(axis=1, by='vsmc_leiden_r05').T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(1.2, 0.95), frameon=False, prop={'size':15})
ax.set_ylabel('Odds Ratio')
plt.tight_layout()
sns.despine(ax=ax)

## Vascular Smooth Muscle Cells (vSMC)

In [ ]:
########################################################################################################################
import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import anndata2ri
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

%%R
library(scran)
library(dplyr)






%config InlineBackend.figure_format = 'retina'

test3_vsmc = anndata.AnnData(X=test3[test3.obs['celltype'].isin(['vSMC.1', 'vSMC.2', 'vSMC.3', 'vSMC.4', 'vSMC.5'])].layers['counts'], obs=test3[test3.obs['celltype'].isin(['vSMC.1', 'vSMC.2', 'vSMC.3', 'vSMC.4', 'vSMC.5'])].obs, var=test3[test3.obs['celltype'].isin(['vSMC.1', 'vSMC.2', 'vSMC.3', 'vSMC.4', 'vSMC.5'])].var)
test3_vsmc.layers["counts"] = test3_vsmc.X.copy()

# Doublet information
#test3_endo.obs['Doublet'] = integrated.obs['Doublet'].loc[test3_endo.obs.index]

# Doublet removal
#test3_endo = test3_endo[test3_endo.obs['Doublet'] == 'False']

adata_pp = test3_vsmc.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.tl.pca(adata_pp, n_comps=15) ## 여기서 이 n_component의 숫자를 늘리면 size_factors를 estimation하는 데 도움이 될까?
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.5)
input_groups = adata_pp.obs['groups']
data_mat = test3_vsmc.X.T
%%R -i data_mat -i input_groups -o size_factors
size_factors = BiocGenerics::sizeFactors(computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups, min.mean=0.1))



del adata_pp
test3_vsmc.obs['size_factors'] = size_factors

test3_vsmc.X /= test3_vsmc.obs['size_factors'].values[:, None]
test3_vsmc.X = scipy.sparse.csr_matrix(test3_vsmc.X) #왜 이게 새로 들어가야될까????? # 아니면 ERRROR 남 (highly_variable_genes에서)

test3_vsmc.layers['scran'] = test3_vsmc.X

sc.pp.log1p(test3_vsmc) # works on anndata.X
#integrated.X = scipy.sparse.csr_matrix(integrated.X)
test3_vsmc.layers['scran_log1p'] = test3_vsmc.X

test3_vsmc.raw = test3_vsmc ## ==> log transforamtion 된 것이 raw로 들어가게 됨.

sc.pp.highly_variable_genes(test3_vsmc)
test3_vsmc.var['highly_variable'].value_counts() # 1,861 ==> 2023-02-27

sc.pp.filter_genes(test3_vsmc, min_cells=0) # integrated.var에 n_cells 추가 ==> test3에서 이루어졌던 n_cells UPDATE

sc.pp.scale(test3_vsmc, max_value=10) # ... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
# adata.raw.X의 mean 과 std를 output함
sc.tl.pca(test3_vsmc, n_comps=100, use_highly_variable=True, svd_solver='arpack')
#sc.pl.pca_variance_ratio(test3_vsmc, n_pcs=100, log=False)

#sce.pp.bbknn default ==> n_pcs=50, neighbors_within_batch=3, trim=None, annoy_n_trees=10,
sce.pp.bbknn(test3_vsmc, batch_key='batch', n_pcs=15, neighbors_within_batch=5, trim=None) #####

sc.tl.umap(test3_vsmc, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
sc.tl.leiden(test3_vsmc, resolution=0.5, key_added='vsmc_leiden_r05')
sc.tl.leiden(test3_vsmc, resolution=1.0, key_added='vsmc_leiden_r10')

test3_vsmc.uns['batch_colors'] = ['#689aff', '#fdbf6f', '#b15928']

np.random.seed(0)
rand_is = np.random.permutation(list(range(test3_vsmc.shape[0])))

sc.pl.umap(test3_vsmc[rand_is, :], add_outline=False, legend_loc='right margin', size=150, color_map='CMRmap', color=['batch','vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'])
sc.pl.umap(test3_vsmc, add_outline=False, legend_loc='right margin', size=150, color_map='CMRmap', color=['batch', 'phase', 'percent_mito'])
sc.pl.umap(test3_vsmc, group_by='Month1', add_outline=False, legend_loc='right margin', size=150, color_map='CMRmap', color=['batch'])


sc.tl.embedding_density(test3_vsmc, basis='umap', groupby='batch')
sc.pl.embedding_density(test3_vsmc, basis='umap', key='umap_density_batch', group=['m01', 'm10', 'm20'])

test3_vsmc.write(filename="/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_vsmc.h5ad")

### Changing name of the vSMC clusters

In [ ]:
test3_vsmc = sc.read_h5ad("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_vsmc.h5ad")

test3_vsmc.uns['vsmc_leiden_r05_colors'] = ['#8dd3c7', '#bebada', '#80b1d3', '#fccde5', '#bc80bd', '#ffed6f']

np.random.seed(0)
rand_is = np.random.permutation(list(range(test3_vsmc.shape[0])))
sc.pl.umap(test3_vsmc[rand_is, :], add_outline=False, legend_loc='right margin', size=50, color=['batch','vsmc_leiden_r05'])

vsmc_leiden_to_celltype_dict = {'0': 'vSMC1',
                                '4': 'vSMC2',
                                '1': 'vSMC3',
                                '2': 'vSMC4',
                                '5': 'vSMC5',
                                '3': 'vSMC6'}

test3_vsmc.obs['Subpopulations of vSMC'] = test3_vsmc.obs['vsmc_leiden_r05'].map(lambda x: vsmc_leiden_to_celltype_dict[x]).astype('category')
order = ('vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'vSMC6')
test3_vsmc.obs['Subpopulations of vSMC'] = test3_vsmc.obs['Subpopulations of vSMC'].cat.reorder_categories(list(order), ordered=True)

test3_vsmc.uns['Subpopulations of vSMC_colors'] = ['#8dd3c7', '#bc80bd', '#bebada', '#80b1d3', '#ffed6f', '#fccde5']

sc.pl.umap(test3_vsmc[rand_is, :], add_outline=False, legend_loc='right margin', size=50, color=['batch','Subpopulations of vSMC'])
sns.despine()

ax = sc.pl.correlation_matrix(test3_vsmc, groupby='Subpopulations of vSMC', show_correlation_numbers=True, dendrogram=True, ax=None, vmin=-1, vmax=1)


##### Choosing Optimal resolution parameter for clustering vSMC using Leiden algorithm

In [ ]:
res_dict = dict()
for res in np.arange(0.3, 1.55, 0.05).round(2):
    temp = sc.tl.leiden(test3_vsmc, resolution=res, copy=True)
    res_dict[res] = len(temp.obs['leiden'].cat.categories)
    
ser = pd.Series(res_dict, name='# of clusters')
ser.index.name = 'Leiden resolution'
df = ser.reset_index()
sns.lineplot(data=df, x='Leiden resolution', y='# of clusters')

# or
from scipy.interpolate import make_interp_spline
lr_new = np.linspace(df['Leiden resolution'].min(), df['Leiden resolution'].max(), 300)
clusters_spline = make_interp_spline(df['Leiden resolution'], df['# of clusters'], k=3)
numclust_new = clusters_spline(lr_new)
plt.plot(lr_new, numclust_new)
plt.xlabel('Leiden Algorithm Resolution')
plt.ylabel('# of vSMC clusters')

#### Stacked Violinplot for various vSMC markers (from *Yap et al., ATVB (2021)*)

In [ ]:
contractile_vsmc_markers = ['Cnn1', 'Smtn', 'Acta2', 'Tagln', 'Myh11', 'Myocd']
mesenchymal_vsmc_markers = ['Klf4', 'Cd34', 'Cd44', 'Ly6a']
#fig, axes = plt.subplots(3, 2, figsize=(7, 10))
#sc.pl.umap(test3_vsmc, color=contractile_vsmc_markers, add_outline=False, legend_loc='right margin', color_map=cmap, show=False, layer='magic', size=80, wspace=1, hspace=1, ax=axes[0])
#axes[0].set_title('Twist1', style='italic')
#sns.despine(ax=axes[0])

plot = sc.pl.StackedViolin(test3_vsmc, contractile_vsmc_markers, groupby='vsmc_leiden_r05', cmap='viridis', figsize=(7,5), return_fig=True, show=False)
plot.legend(title='Median Expression\nin each cluster')
plot.style(cmap='viridis', stripplot=True, jitter=True)
plot.swap_axes(swap_axes=True)
plot.get_axes()['mainplot_ax'].set_xticklabels(labels=['0', '1', '2', '3', '4', '5'], rotation=0)
plot.get_axes()['mainplot_ax'].set_yticklabels(labels=ec1_markers, fontstyle='italic')
plot.get_axes()['mainplot_ax'].set_xlabel("Subpopulations of Endothelial Cells")
sns.despine(ax=plot.get_axes()['mainplot_ax'], left=False, bottom=False)

axes[0].text(-8.5, 8, "G", size=20, weight='bold')

#### Differential Expression Testing (Wilcoxon Rank-sum test)

In [ ]:
sc.tl.rank_genes_groups(test3_vsmc, 'vsmc_leiden_r05', method='wilcoxon', use_raw=True, pts=True, key_added='vsmc_leiden_r05_rank_genes_groups')
# use_raw=True => scran_log1p
#sc.pl.rank_genes_groups(test3_vsmc, n_genes=10, sharey=False)
sc.pl.rank_genes_groups_heatmap(test3_vsmc, n_genes=10, min_logfoldchange=2, cmap='cividis', show_gene_labels=True, key='vsmc_leiden_r05_rank_genes_groups')

deg_vsmc_result = test3_vsmc.uns['vsmc_leiden_r05_rank_genes_groups']
#deg_vsmc_result.keys() #dict_keys(['params', 'pts', 'pts_rest', 'names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])


deg_vsmc_groups = deg_vsmc_result['names'].dtype.names
deg_vsmc_wilcoxon = pd.DataFrame({group + '_' + key: deg_vsmc_result[key][group] for group in deg_vsmc_groups for key in ['names', 'logfoldchanges', 'scores', 'pvals_adj', 'pts', 'pts_rest']})




#### Differential Expression Testing with respect to aging (Wilcoxon Rank-sum test)

In [ ]:
sc.tl.rank_genes_groups(test3_vsmc, 'batch', method='wilcoxon', use_raw=True, pts=True, key_added='batch_rank_genes_groups')
# use_raw=True => scran_log1p
#sc.pl.rank_genes_groups(test3_vsmc, n_genes=10, sharey=False)
sc.pl.rank_genes_groups_heatmap(test3_vsmc, n_genes=10, min_logfoldchange=2, cmap='cividis', show_gene_labels=True, key='batch_rank_genes_groups')

deg_vsmc_result = test3_vsmc.uns['vsmc_leiden_r05_rank_genes_groups']
#deg_vsmc_result.keys() #dict_keys(['params', 'pts', 'pts_rest', 'names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])


deg_vsmc_groups = deg_vsmc_result['names'].dtype.names
deg_vsmc_wilcoxon = pd.DataFrame({group + '_' + key: deg_vsmc_result[key][group] for group in deg_vsmc_groups for key in ['names', 'logfoldchanges', 'scores', 'pvals_adj', 'pts', 'pts_rest']})




#### MAST Differential Expression Testing

#### Gene-set enrichment analysis (Wilcoxon Rank-sum test)

In [ ]:
import gseapy as gp
gp.get_library_name(organism='Mouse')

vsmc_cluster0_log2fc_1up = list(deg_vsmc_wilcoxon[(deg_vsmc_wilcoxon['0_pvals_adj'] < 0.0001) & (deg_vsmc_wilcoxon['0_logfoldchanges'] > 1)]['0_names'].values)
vsmc_cluster0_log2fc_1down = list(deg_vsmc_wilcoxon[(deg_vsmc_wilcoxon['0_pvals_adj'] < 0.0001) & (deg_vsmc_wilcoxon['0_logfoldchanges'] < -1)]['0_names'].values)

total_no_vsmc = test3_vsmc.shape[0] # number of cells in vSMC cluster (test3_vsmc) => for identifying expressed genes which later be used as backgroud gene set!!
background_genesets_vsmc = test3_vsmc.var[(test3_vsmc.var['n_cells'] > total_no_vsmc * 0.005)] # amounts to 0.5% of total vSMC cells
# 위는 좀 생각할 필요가 있음. 왜냐하면 vsmc를 할 때 EC를 할 때 FB를 할 때 모두 다 다른 background geneset을 가질 것이기 때문
# 따라서, 아예 test3 (whole cells)에서 몇 개 이상의 cell 에서 expression되는 유전자들을 다 가져온다음에 얘를 대상으로 해야할 수도

go_vsmc_cluster0_log2fc_1up = gp.enrichr(gene_list=vsmc_cluster0_log2fc_1up,
                                         gene_sets=['GO_Biological_Process_2021'],
                                         background=background_genesets_vsmc,
                                         organism='Mouse',
                                         outdir=None,
                                         cutoff=0.05,
                                         no_plot=True) # 이거 background option이 있는데 (?gp.enrichr) 확인 꼭 해봐야 함!
go_vsmc_cluster0_log2fc_1up.res2d.Term = go_vsmc_cluster0_log2fc_1up.res2d.Term.str.split(" \(GO").str[0]


go_vsmc_cluster0_log2fc_1down = gp.enrichr(gene_list=vsmc_cluster0_log2fc_1down,
                                         gene_sets=['GO_Biological_Process_2021'],
                                         background=background_genesets_vsmc,
                                         organism='Mouse',
                                         outdir=None,
                                         cutoff=0.05,
                                         no_plot=True)
go_vsmc_cluster0_log2fc_1down.res2d.Term = go_vsmc_cluster0_log2fc_1down.res2d.Term.str.split(" \(GO").str[0]

gp.dotplot(go_vsmc_cluster0_log2fc_2up.res2d, 
           figsize=(3, 5), 
           size=200, 
           title="vSMC Cluster_0 UP (log2FC>2)")

# concat results
go_vsmc_cluster0_log2fc_1up.res2d['UP_DW'] = "UP"
go_vsmc_cluster0_log2fc_1down.res2d['UP_DW'] = "DOWN"
go_vsmc_cluster0_res = pd.concat([go_vsmc_cluster0_log2fc_1up.res2d.head(), go_vsmc_cluster0_log2fc_1down.res2d.head()]) # 각각 5개씩만


ax = gp.barplot(go_vsmc_cluster0_res,
                group=["UP_DW"],
                title='vSMC Cluster_0 UP (log2FC>2) DOWN (log2FC<-2)',
                column='Adjusted P-value',
                color= ['b', 'r'],
                figsize=(15,10))
plt.tight_layout()

ax = gp.dotplot(go_vsmc_cluster0_log2fc_2up.res2d, 
                figsize=(10, 10), 
                size=100, 
                cmap='NbDr_r',
                show_ring=True,
                title="vSMC Cluster_0 UP (log2FC>2) DOWN (log2FC<-2)")

#ofname='./GO_EC_subclsters/Barplot_EC_1_GO_Biological_Process.pdf')

#### Table 화

#### Fold enrichment (Odds ratio) of vSMC subpopulation for each age group

In [ ]:
df = test3_vsmc.obs[['batch', 'Subpopulations of vSMC']]
df_pivot = pd.crosstab(df['batch'], df['Subpopulations of vSMC'], normalize=False, margins=True)

# Percentage (intracluster normalization) heatmap 
sns.heatmap(pd.crosstab(df['batch'], df['Subpopulations of vSMC'], margins=False, normalize='columns')*100, vmin=0, vmax=100, cmap='viridis', annot=True, fmt='.2f')
plt.tight_layout()

# Odds ratio calculation
oddsratio_df, pvalue_df = list(), list()
for month in df_pivot.index[:-1]:
    oddsratio, pvalues = list(), list()
    for celltype in df_pivot.columns[:-1]:
#        table = np.array([ [df_pivot[celltype][month], df_pivot[celltype]['All'] - df_pivot[celltype][month] ], [df_pivot['All'][month], df_pivot['All']['All'] - df_pivot['All'][month]] ])
        table = np.array([ [df_pivot[celltype][month], df_pivot['All'][month] - df_pivot[celltype][month]], [df_pivot[celltype]['All'] - df_pivot[celltype][month], (df_pivot['All']['All'] - df_pivot['All'][month]) - (df_pivot[celltype]['All'] - df_pivot[celltype][month])] ])
        oddsr, p = fisher_exact(table, alternative='two-sided')
        oddsratio.append(oddsr)
        pvalues.append(p)
    oddsratio_df.append(oddsratio)
    pvalues = multipletests(pvals=pvalues, alpha=0.01, method='fdr_bh')[1]
    pvalue_df.append(pvalues)

Odds = pd.DataFrame(oddsratio_df, index=df_pivot.index[:-1], columns=df_pivot.columns[:-1])
Pvalues = pd.DataFrame(pvalue_df, index=df_pivot.index[:-1], columns=df_pivot.columns[:-1])

df_final = pd.concat([Odds, Pvalues], axis=0)
df_final

batch_palette= {'m01': '#689aff',
                'm10': '#fdbf6f',
                'm20': '#b15928'}

ax = df_final.iloc[:3].sort_values(axis=1, by='Subpopulations of vSMC').T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(0.71, 0.95), frameon=False, prop={'size':15})
ax.set_ylabel('Odds Ratio')
ax.axhline(y=1.0, color="red", linewidth=1.0, linestyle='--')
plt.tight_layout()
#sns.despine(ax=ax)

#plt.savefig("FigureD.pdf")

# Scale [0,1] using min-max normalization for each vSMC subcluster
df_odds_scale = (df_final.iloc[:3] - df_final.iloc[:3].min()) / ( df_final.iloc[:3].max() - df_final.iloc[:3].min() )
ax = df_odds_scale.sort_values(axis=1, by='Subpopulations of vSMC').T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(1.2, 0.95), frameon=False, prop={'size':15})
ax.set_ylabel('Scaled Odds Ratio')
plt.tight_layout()
sns.despine(ax=ax)

### Diffusion Pseudotime of vSMC

In [ ]:
#test3_vsmc = sc.read_h5ad("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_vsmc.h5ad")

sc.tl.diffmap(test3_vsmc)
sc.pl.diffmap(test3_vsmc[rand_is, :], add_outline=False, legend_loc='right margin', size=40, color_map='CMRmap', color=['batch','Subpopulations of vSMC'])

start_cell = np.isin(test3_vsmcfb.obs['endo_leiden_r05'], '0') # boolean numpy array ==> array([False, False, False, ..., False, False, False])
#max_start_id = np.argmin(test3_endo.obsm['X_diffmap'][start_cell,1]) # 262
max_start_id = np.argmax(test3_vsmcfb.obsm['X_diffmap'][start_cell,1])
root_id = np.arange(len(start_cell))[start_cell][max_start_id] # 341
test3_vsmcfb.uns['iroot'] = root_id

sc.tl.dpt(test3_vsmcfb, n_branchings=1, n_dcs=10) # n_branchings를 0으로 하면 (recommended by Scanpy developer) dpt_groups가 생성 안 됨.
#computing Diffusion Pseudotime using n_dcs=10
sc.pl.dpt_groups_pseudotime(test3_vsmcfb) # 여기에서 pseudotime trajecgory 확인.

lin = ('2', '0', '3', '1') # DPT pseudotime group ordering에 맞게 배치
test3_vsmcfb.obs['dpt_groups'] = test3_vsmcfb.obs['dpt_groups'].cat.reorder_categories(list(lin), ordered=True)
sc.pl.dpt_groups_pseudotime(test3_vsmcfb) # 다시 ordering에 맞게 plotting
sc.pl.dpt_timeseries(test3_vsmcfb[:, test3_vsmcfb.var.highly_variable])

## Vascular Smooth muscel cell and Fibroblast combined

In [ ]:
test3 = sc.read_h5ad("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3.h5ad")

leiden_to_consensus_celltype_dict = {'0': 'vSMC',
'1': 'vSMC',
'2': 'vSMC',
'3': 'FB',
'4': 'vSMC',
'5': 'EC',
'6': 'FB',
'7': 'EC',
'8': 'vSMC',
'9': 'FB',
'10': 'Bc',
'11': 'M\u03A6',
'12': 'Tc',
'13': 'Neuronal'}
test3.obs['Consensus_cell_type'] = test3.obs['leiden_r05'].map(lambda x: leiden_to_consensus_celltype_dict[x]).astype('category')
consensus_celltype_order = ('EC', 'vSMC', 'FB', 'Bc', 'M\u03A6', 'Tc', 'Neuronal')
test3.obs['Consensus_cell_type'] = test3.obs['Consensus_cell_type'].cat.reorder_categories(list(consensus_celltype_order), ordered=True)

leiden_to_detailed_celltype_dict = {'0': 'vSMC1',
'1': 'vSMC2',
'2': 'vSMC3',
'3': 'FB1',
'4': 'vSMC4',
'5': 'EC1',
'6': 'FB2',
'7': 'EC2',
'8': 'vSMC5',
'9': 'FB3',
'10': 'Bc',
'11': 'Mphage',
'12': 'Tc',
'13': 'Neuronal'}
test3.obs['detailed_celltype'] = test3.obs['leiden_r05'].map(lambda x: leiden_to_detailed_celltype_dict[x]).astype('category')
detailed_celltype_order = ('EC1', 'EC2', 'vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3', 'Bc', 'Mphage', 'Tc', 'Neuronal')
test3.obs['detailed_celltype'] = test3.obs['detailed_celltype'].cat.reorder_categories(list(detailed_celltype_order), ordered=True)


########################################################################################################################
import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import anndata2ri
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

%%R
library(scran)
library(dplyr)






%config InlineBackend.figure_format = 'retina'

test3_vsmcfb = anndata.AnnData(X=test3[test3.obs['detailed_celltype'].isin(['vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3'])].layers['counts'], obs=test3[test3.obs['detailed_celltype'].isin(['vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3'])].obs, var=test3[test3.obs['detailed_celltype'].isin(['vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3'])].var)
test3_vsmcfb.layers["counts"] = test3_vsmcfb.X.copy()

# Doublet information
#test3_endo.obs['Doublet'] = integrated.obs['Doublet'].loc[test3_endo.obs.index]

# Doublet removal
#test3_endo = test3_endo[test3_endo.obs['Doublet'] == 'False']

adata_pp = test3_vsmcfb.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.tl.pca(adata_pp, n_comps=15) ## 여기서 이 n_component의 숫자를 늘리면 size_factors를 estimation하는 데 도움이 될까?
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.5)
input_groups = adata_pp.obs['groups']
data_mat = test3_vsmcfb.X.T
%%R -i data_mat -i input_groups -o size_factors
size_factors = BiocGenerics::sizeFactors(computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups, min.mean=0.1))



del adata_pp
test3_vsmcfb.obs['size_factors'] = size_factors

test3_vsmcfb.X /= test3_vsmcfb.obs['size_factors'].values[:, None]
test3_vsmcfb.X = scipy.sparse.csr_matrix(test3_vsmcfb.X) #왜 이게 새로 들어가야될까????? # 아니면 ERRROR 남 (highly_variable_genes에서)

test3_vsmcfb.layers['scran'] = test3_vsmcfb.X

sc.pp.log1p(test3_vsmcfb) # works on anndata.X
#integrated.X = scipy.sparse.csr_matrix(integrated.X)
test3_vsmcfb.layers['scran_log1p'] = test3_vsmcfb.X

test3_vsmcfb.raw = test3_vsmcfb ## ==> log transforamtion 된 것이 raw로 들어가게 됨.

sc.pp.highly_variable_genes(test3_vsmcfb)
test3_vsmcfb.var['highly_variable'].value_counts() # 1,981 ==> 2023-03-14

sc.pp.filter_genes(test3_vsmcfb, min_cells=0) # integrated.var에 n_cells 추가 ==> test3에서 이루어졌던 n_cells UPDATE

sc.pp.scale(test3_vsmcfb, max_value=10) # ... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
# adata.raw.X의 mean 과 std를 output함
sc.tl.pca(test3_vsmcfb, n_comps=100, use_highly_variable=True, svd_solver='arpack')
sc.pl.pca_variance_ratio(test3_vsmcfb, n_pcs=100, log=False)
sc.pl.pca_loadings(test3_vsmcfb, components=[1, 2, 3, 4])

#sce.pp.bbknn default ==> n_pcs=50, neighbors_within_batch=3, trim=None, annoy_n_trees=10,
sce.pp.bbknn(test3_vsmcfb, batch_key='batch', n_pcs=20, neighbors_within_batch=3, trim=None) #####
#sce.pp.bbknn(test3_vsmcfb, batch_key='batch', n_pcs=20, neighbors_within_batch=5, trim=None)
#sce.pp.bbknn(test3_vsmcfb, batch_key='batch', n_pcs=15, neighbors_within_batch=10, trim=None) # Absurd
#sce.pp.bbknn(test3_vsmcfb, batch_key='batch', n_pcs=15, neighbors_within_batch=3, trim=None)

sc.tl.umap(test3_vsmcfb, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
sc.tl.leiden(test3_vsmcfb, resolution=0.5, key_added='vsmcfb_leiden_r05')
sc.tl.leiden(test3_vsmcfb, resolution=1.0, key_added='vsmcfb_leiden_r10')

test3_vsmcfb.uns['batch_colors'] = ['#689aff', '#fdbf6f', '#b15928']

np.random.seed(0)
rand_is = np.random.permutation(list(range(test3_vsmcfb.shape[0])))
sc.pl.umap(test3_vsmcfb[rand_is,:], add_outline=False, legend_loc='right margin', size=40, color_map=cmap, color=['batch','vsmcfb_leiden_r05', 'vsmcfb_leiden_r10', 'detailed_celltype'])

# There are numerous controversy that non-linear projection (e.g. UMAP/t-SNE) fails to represent true (in a sense) topological nature of the high-dimensional data.
# So it's best to first detect clusters (based on leiden clustering for example) and then look into those cluster using conventional similiarity measure such as hierarchical clustering 

sc.pl.umap(test3_vsmcfb[rand_is,:], add_outline=False, legend_loc='right margin', size=40, color_map=cmap, color=['batch', 'phase', 'percent_mito'])
sc.pl.umap(test3_vsmcfb[rand_is,:], group_by='Month1', add_outline=False, legend_loc='right margin', size=40, color_map='CMRmap', color=['batch'])

vsmcfb_leiden_to_celltype_dict = {'5': 'vSMC1',
                                '0': 'vSMC2',
                                '1': 'vSMC3',
                                '4': 'vSMC4',
                                '3': 'vSMC5',
                                '2': 'FB1',
                                '6': 'FB2',
                                '7': 'FB3'}

test3_vsmcfb.obs['Subpopulations of vSMC and FB'] = test3_vsmcfb.obs['vsmcfb_leiden_r05'].map(lambda x: vsmcfb_leiden_to_celltype_dict[x]).astype('category')
order = ('vSMC1', 'vSMC2', 'vSMC3', 'vSMC4', 'vSMC5', 'FB1', 'FB2', 'FB3')
test3_vsmcfb.obs['Subpopulations of vSMC and FB'] = test3_vsmcfb.obs['Subpopulations of vSMC and FB'].cat.reorder_categories(list(order), ordered=True)

#test3_vsmcfb.write(filename="/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_vsmcfb.h5ad")

#### Diffmap and pseudotime for vSMC+FB

In [2]:
test3_vsmcfb = sc.read_h5ad("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_vsmcfb.h5ad")

sc.tl.diffmap(test3_vsmcfb)
sc.pl.diffmap(test3_vsmcfb[rand_is, :], add_outline=False, legend_loc='right margin', size=40, color_map='CMRmap', color=['batch','Subpopulations of vSMC and FB'])

start_cell = np.isin(test3_vsmcfb.obs['endo_leiden_r05'], '0') # boolean numpy array ==> array([False, False, False, ..., False, False, False])
#max_start_id = np.argmin(test3_endo.obsm['X_diffmap'][start_cell,1]) # 262
max_start_id = np.argmax(test3_vsmcfb.obsm['X_diffmap'][start_cell,1])
root_id = np.arange(len(start_cell))[start_cell][max_start_id] # 341
test3_vsmcfb.uns['iroot'] = root_id

sc.tl.dpt(test3_vsmcfb, n_branchings=1, n_dcs=10) # n_branchings를 0으로 하면 (recommended by Scanpy developer) dpt_groups가 생성 안 됨.
#computing Diffusion Pseudotime using n_dcs=10
sc.pl.dpt_groups_pseudotime(test3_vsmcfb) # 여기에서 pseudotime trajecgory 확인.

lin = ('2', '0', '3', '1') # DPT pseudotime group ordering에 맞게 배치
test3_vsmcfb.obs['dpt_groups'] = test3_vsmcfb.obs['dpt_groups'].cat.reorder_categories(list(lin), ordered=True)
sc.pl.dpt_groups_pseudotime(test3_vsmcfb) # 다시 ordering에 맞게 plotting
sc.pl.dpt_timeseries(test3_vsmcfb[:, test3_vsmcfb.var.highly_variable])


### ForceAtlas Graph

In [ ]:
sc.tl.draw_graph(test3_vsmcfb, layout='fa')
sc.pl.draw_graph(test3_vsmcfb, color=['batch', 'Subpopulations of vSMC and FB'], add_outline=True, legend_loc='right margin', size=10, color_map='CMRmap')


#### Diffmap and pseudotime for vSMC

In [ ]:
sc.tl.diffmap(test3_vsmc)
sc.pl.diffmap(test3_vsmc, add_outline=False, legend_loc='right margin', size=70, color_map='CMRmap', color=['batch','vsmc_leiden_r05'])


In [ ]:
sce.pp.bbknn(test3_vsmc, batch_key='batch', n_pcs=10, neighbors_within_batch=5, trim=None)
sc.tl.umap(test3_vsmc, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
sc.tl.leiden(test3_vsmc, resolution=0.5, key_added='vsmc_leiden_r05')
sc.tl.leiden(test3_vsmc, resolution=1.0, key_added='vsmc_leiden_r10')
sc.pl.umap(test3_vsmc, add_outline=False, legend_loc='right margin', size=150, color_map=cmap, title=['npc10_nwb5', 'vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'], color=['batch','vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'])

sce.pp.bbknn(test3_vsmc, batch_key='batch', n_pcs=20, neighbors_within_batch=5, trim=None)
sc.tl.umap(test3_vsmc, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
sc.tl.leiden(test3_vsmc, resolution=0.5, key_added='vsmc_leiden_r05')
sc.tl.leiden(test3_vsmc, resolution=1.0, key_added='vsmc_leiden_r10')
sc.pl.umap(test3_vsmc, add_outline=False, legend_loc='right margin', size=150, color_map=cmap, title=['npc20_nwb5', 'vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'], color=['batch','vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'])

sce.pp.bbknn(test3_vsmc, batch_key='batch', n_pcs=25, neighbors_within_batch=5, trim=None)
sc.tl.umap(test3_vsmc, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
sc.tl.leiden(test3_vsmc, resolution=0.5, key_added='vsmc_leiden_r05')
sc.tl.leiden(test3_vsmc, resolution=1.0, key_added='vsmc_leiden_r10')
sc.pl.umap(test3_vsmc, add_outline=False, legend_loc='right margin', size=150, color_map=cmap, title=['npc25_nwb5', 'vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'], color=['batch','vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'])

sce.pp.bbknn(test3_vsmc, batch_key='batch', n_pcs=30, neighbors_within_batch=5, trim=None)
sc.tl.umap(test3_vsmc, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
sc.tl.leiden(test3_vsmc, resolution=0.5, key_added='vsmc_leiden_r05')
sc.tl.leiden(test3_vsmc, resolution=1.0, key_added='vsmc_leiden_r10')
sc.pl.umap(test3_vsmc, add_outline=False, legend_loc='right margin', size=150, color_map=cmap, title=['npc30_nwb5', 'vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'], color=['batch','vsmc_leiden_r05', 'vsmc_leiden_r10', 'celltype'])




### FB analysis

In [ ]:
test3_fb = sc.read_h5ad("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_fb.h5ad")

sc.tl.leiden(test3_fb, resolution=0.25, key_added='fb_leiden_r025') #2023-04-13

sc.tl.embedding_density(test3_fb, basis='umap', groupby='batch')
sc.pl.embedding_density(test3_fb, basis='umap', key='umap_density_batch', group=['m01', 'm10', 'm20'])

test3_fb.uns['vsmc_leiden_r05_colors'] = ['#8dd3c7', '#bebada', '#80b1d3', '#fccde5', '#bc80bd', '#ffed6f']

# To color cells in random way
np.random.seed(0)
rand_is = np.random.permutation(list(range(test3_fb.shape[0])))
sc.pl.umap(test3_fb[rand_is, :], add_outline=False, legend_loc='right margin', size=50, color=['batch','fb_leiden_r05'])

fb_leiden_to_celltype_dict = {'7': 'FB1',
                              '1': 'FB2',
                              '5': 'FB3',
                              '2': 'FB4',
                              '0': 'FB5',
                              '4': 'FB6',
                              '3': 'FB7',
                              '6': 'FB8',
                              '8': 'FB9'}

test3_fb.obs['Subpopulations of FB'] = test3_fb.obs['fb_leiden_r05'].map(lambda x: fb_leiden_to_celltype_dict[x]).astype('category')
order = ('FB1', 'FB2', 'FB3', 'FB4', 'FB5', 'FB6', 'FB7', 'FB8', 'FB9')
test3_fb.obs['Subpopulations of FB'] = test3_fb.obs['Subpopulations of FB'].cat.reorder_categories(list(order), ordered=True)

test3_fb.uns['Subpopulations of vSMC_colors'] = ['#8dd3c7', '#bc80bd', '#bebada', '#80b1d3', '#ffed6f', '#fccde5']

np.random.seed(0)
rand_is = np.random.permutation(list(range(test3_fb.shape[0])))
sc.pl.umap(test3_fb[rand_is, :], add_outline=False, legend_loc='on data', size=50, color=['batch','Subpopulations of FB'])


ax = sc.pl.correlation_matrix(test3_fb, groupby='Subpopulations of FB', show_correlation_numbers=True, dendrogram=True, ax=None, vmin=-1, vmax=1)

sc.tl.draw_graph(test3_fb, layout='fa')
sc.pl.draw_graph(test3_fb[rand_is, :], add_outline=False, legend_loc='on data', size=50, color=['batch', 'Subpopulations of FB', 'fb_leiden_r025'])


#### Fold enrichment of FB population

In [ ]:
df = test3_fb.obs[['batch', 'Subpopulations of FB']]
df_pivot = pd.crosstab(df['batch'], df['Subpopulations of FB'], normalize=False, margins=True)

# Percentage (intracluster normalization) heatmap 
sns.heatmap(pd.crosstab(df['batch'], df['Subpopulations of FB'], margins=False, normalize='columns')*100, vmin=0, vmax=100, cmap='viridis', annot=True, fmt='.2f')
plt.tight_layout()

# Odds ratio calculation
oddsratio_df, pvalue_df = list(), list()
for month in df_pivot.index[:-1]:
    oddsratio, pvalues = list(), list()
    for celltype in df_pivot.columns[:-1]:
#        table = np.array([ [df_pivot[celltype][month], df_pivot[celltype]['All'] - df_pivot[celltype][month] ], [df_pivot['All'][month], df_pivot['All']['All'] - df_pivot['All'][month]] ])
        table = np.array([ [df_pivot[celltype][month], df_pivot['All'][month] - df_pivot[celltype][month]], [df_pivot[celltype]['All'] - df_pivot[celltype][month], (df_pivot['All']['All'] - df_pivot['All'][month]) - (df_pivot[celltype]['All'] - df_pivot[celltype][month])] ])
        oddsr, p = fisher_exact(table, alternative='two-sided')
        oddsratio.append(oddsr)
        pvalues.append(p)
    oddsratio_df.append(oddsratio)
    pvalues = multipletests(pvals=pvalues, alpha=0.01, method='fdr_bh')[1]
    pvalue_df.append(pvalues)

Odds = pd.DataFrame(oddsratio_df, index=df_pivot.index[:-1], columns=df_pivot.columns[:-1])
Pvalues = pd.DataFrame(pvalue_df, index=df_pivot.index[:-1], columns=df_pivot.columns[:-1])

df_final = pd.concat([Odds, Pvalues], axis=0)
df_final

batch_palette= {'m01': '#689aff',
                'm10': '#fdbf6f',
                'm20': '#b15928'}

ax = df_final.iloc[:3].sort_values(axis=1, by='Subpopulations of FB').T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(0.71, 1.03), frameon=False, prop={'size':15})
ax.set_ylabel('Odds Ratio')
ax.axhline(y=1.0, color="red", linewidth=1.0, linestyle='--')
plt.tight_layout()
#sns.despine(ax=ax)

#plt.savefig("FigureD.pdf")

# Scale [0,1] using min-max normalization for each vSMC subcluster
df_odds_scale = (df_final.iloc[:3] - df_final.iloc[:3].min()) / ( df_final.iloc[:3].max() - df_final.iloc[:3].min() )
ax = df_odds_scale.sort_values(axis=1, by='Subpopulations of FB').T.plot.bar(color=batch_palette, rot=0) # Odds ratio
ax.legend(loc='upper left', bbox_to_anchor=(1.2, 0.95), frameon=False, prop={'size':15})
ax.set_ylabel('Scaled Odds Ratio')
plt.tight_layout()
sns.despine(ax=ax)

# Trajectory Analysis

In [ ]:
# AnnData to SingelCellExperiment/Seurat object
from scipy import io
io.mmwrite('/mnt/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_fb_scran.mtx', test3_fb.layers['scran'])
io.mmwrite('/mnt/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_fb_counts.mtx', test3_fb.layers['counts'])
cell_meta = test3_fb.obs[['batch', 'fb_leiden_r05', 'fb_leiden_r025']].copy()
cell_meta['Barcode'] = cell_meta.index
cell_meta['UMAP1'] = test3_fb.obsm['X_umap'][:,0]
cell_meta['UMAP2'] = test3_fb.obsm['X_umap'][:,1]
cell_meta['FA1'] = test3_fb.obsm['X_draw_graph_fa'][:,0]
cell_meta['FA2'] = test3_fb.obsm['X_draw_graph_fa'][:,1]
gene_meta = test3_fb.var.copy()
gene_meta['GeneName'] = gene_meta.index
cell_meta.to_csv('/mnt/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_fb_cellMeta.csv',index=None)
gene_meta.to_csv('/mnt/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/test3_fb_geneMeta.csv',index=None)

rfh = open('test3_fb_hvg_genelist.txt', 'w')

for gene in list(test3_fb.var[test3_fb.var['highly_variable'] == True].index):
    rfh.write(gene + '\n')
    rfh.flush()

rfh.close()

# From bdcm03 scRNA_R3 conda environment
'''
library(Seurat)
library(Matrix)

counts <- readMM('/mnt/mone/Project/WC300/Vascular_Aging/FB/test3_fb_counts.mtx')
dim(counts)
cellMeta<-read.csv('/mnt/mone/Project/WC300/Vascular_Aging/FB/test3_fb_cellMeta.csv')
geneMeta<-read.csv('/mnt/mone/Project/WC300/Vascular_Aging/FB/test3_fb_geneMeta.csv')
rownames(counts) <- cellMeta$Barcode
colnames(counts) <- geneMeta$GeneName
seurat_obj <- CreateSeuratObject(counts = t(counts), min.cells = 0, min.features = 0, names.field = 1, names.delim = "_")
seurat_obj@meta.data <- cbind(cellMeta, seurat_obj@meta.data)
seurat_obj <- FindVariableFeatures(seurat_obj, selection.method = "vst", nfeatures = 3000, verbose = TRUE)
seurat_obj <- ScaleData(seurat_obj, verbose = TRUE) # seurat_obj[["RNA"]]@scale.data, ORIGINAL: seurat_obj[["RNA"]]@data
seurat_obj <- RunPCA(seurat_obj, features = VariableFeatures(object = seurat_obj), verbose = TRUE)
seurat_obj <- RunUMAP(seurat_obj, dims = 1:15, verbose = TRUE)

#seo@reductions$umap2 <- seo@reductions$umap
#head(seo@reductions$umap@cell.embeddings)

scanpy_umap <- seurat_obj@meta.data[, c('UMAP1','UMAP2')]
scanpy_fa <- seurat_obj@meta.data[, c('FA1','FA2')]
colnames(scanpy_umap) <- c('UMAP_1','UMAP_2')
colnames(scanpy_fa) <- c('FA_1','FA_2')
seurat_obj@reductions$umap@cell.embeddings <- as.matrix(scanpy_umap)
seurat_obj@reductions$fa@cell.embeddings <- as.matrix(scanpy_fa)



suppressMessages(library(Seurat))
suppressMessages(library(BiocParallel))
suppressMessages(library(slingshot))
suppressMessages(library(tradeSeq))
suppressMessages(library(scales))
suppressMessages(library(viridis))
suppressMessages(library(RColorBrewer))

# Assign colors
ggplotColours <- function(n = 6, h = c(0, 360) + 15){
  if ((diff(h) %% 360) < 1) h[2] <- h[2] - 360/n
  hcl(h = (seq(h[1], h[2], length = n)), c = 100, l = 65)
}

color_list <- ggplotColours(n=14)

fb_sce <- as.SingleCellExperiment(seurat_obj)
fb_sce$fb_leiden_r05 <- as.character(fb_sce$fb_leiden_r05)
fb_sce$fb_leiden_r025 <- as.character(fb_sce$fb_leiden_r025)
fb_sce <- slingshot(fb_sce, clusterLabels = 'fb_leiden_r05', reducedDim = 'UMAP', start.clus = "0")
fb_sce <- slingshot(fb_sce, clusterLabels = 'fb_leiden_r05', reducedDim = 'UMAP', start.clus = "0", end.clus = "7")
fb_sce <- slingshot(fb_sce, clusterLabels = 'fb_leiden_r05', reducedDim = 'UMAP', start.clus = "0", end.clus = "6")
fb_sce <- slingshot(fb_sce, clusterLabels = 'fb_leiden_r05', reducedDim = 'UMAP')
#SlingshotDataSet(fb_sce)

cell_pal <- function(cell_vars, pal_fun,...) {
  if (is.numeric(cell_vars)) {
    pal <- pal_fun(100, ...)
    return(pal[cut(cell_vars, breaks = 100)])
  } else {
    categories <- sort(unique(cell_vars))
    pal <- setNames(pal_fun(length(categories), ...), categories)
    return(pal[cell_vars])
  }
}

cell_colors_clust <- cell_pal(fb_sce$fb_leiden_r05, hue_pal())

pdf("UMAP_wSlingshot_default.pdf")
plot(reducedDim(fb_sce, "UMAP"), pch = 16, cex = 0.5)
lines(SlingshotDataSet(fb_sce), lwd = 2, type = 'lineages', col = 'gray')
dev.off()

pdf("UMAP_wSlingshot_clusters.pdf")
plot(reducedDim(fb_sce, "UMAP"), col = cell_colors_clust, pch = 16, cex = 0.5, show.constraints=TRUE)
lines(SlingshotDataSet(fb_sce), lwd = 2, type = 'lineages', col = 'black')
dev.off()

pdf("UMAP_wSlingshotcurves_clusters.pdf")
plot(reducedDim(fb_sce, "UMAP"), col = cell_colors_clust, pch = 16, cex = 0.5, show.constraints=TRUE)
lines(SlingshotDataSet(fb_sce), lwd = 2, type = 'curves', col = 'black')
dev.off()

pdf("UMAP_wSlingshotcurves_Different_Lineages.pdf", width=7, height=8)
nc <- 2
pt <- slingPseudotime(fb_sce)
nms <- colnames(pt)
nr <- ceiling(length(nms)/nc)
pal <- viridis(100, end = 0.95)
par(mfrow = c(nr, nc))
c <- 0
for (i in nms) {
  c <- c + 1
  colors <- pal[cut(pt[,i], breaks = 100)]
  plot(reducedDim(fb_sce, "UMAP"), col = colors, pch = 16, cex = 0.35, main = i)
  lines(SlingshotDataSet(fb_sce), lwd = 2, col = 'black', type = 'curves', linInd=c)
}
dev.off()

fitgam_fb_sce <- fitGAM(assays(altExp(fb_sce))$logcounts[rownames(fb_sce),], sds=SlingshotDataSet(fb_sce), nknots=10)
assoRes <- associationTest(fitgam_fb_sce, contrastType="end", l2fc=0)
startRes <- startVsEndTest(fitgam_fb_sce)
oStart <- order(startRes$waldStat, decreasing = TRUE)
#head(startRes[order(startRes$waldStat, decreasing=TRUE),])

plotSmoothers(fitgam_cd8_sce, assays(altExp(cd8_sce))$logcounts[rownames(cd8_sce),], gene = sigGeneStart)

plotGeneCount(SlingshotDataSet(cd8_sce), counts=assays(altExp(cd8_sce))$logcounts[rownames(cd8_sce),], gene = sigGeneStart)



patternRes <- patternTest(fitgam_cd8_sce)
oPat <- order(patternRes$waldStat, decreasing = TRUE)
#head(rownames(patternRes)[oPat])

pdf("IL7R.pdf")
plotSmoothers(fitgam_cd8_sce, assays(altExp(cd8_sce))$logcounts[rownames(cd8_sce),], gene = rownames(patternRes)[oPat][1])

'''
